<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf


    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance


    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-10 13:19:13
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  9.00 K
Current:  1.37 C
-------------------
Today PnL: -11.50 K (-0.08%)
Current PnL: -13.11 L (-9.14%)
CY Booked + Current PnL: -6.15 L (-4.29%)
-------------------
Total profit:  4.82 L
Total loss:  -17.94 L
-------------------
Total Booked + Current PnL: 20.83 L (17.87%)
Total Booked PnL: 33.94 L (29.12%)
Curr Year Booked PnL: 6.96 L (5.08%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 71.60 L (52.26%)
Deployed:  1.17 C
Current:  1.37 C
CAGR/XIRR %: 10.87%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.00,-221.42,H-SC,98.96,87545.0,-7771.0,7809.0,2.49,-8.15,8.92,0.04,157.0,-1.00,0.64,72.67,OX40N,NTT,MISC
4,APOLLOHOSP,6904.43,8285.00,-18.29,H-LC,90.62,111248.0,7682.0,13027.0,-0.70,7.42,11.71,20.00,32.0,0.59,0.81,22.53,X40N,BTT,HEALTHCARE
39,HINDALCO,651.95,761.55,-21.55,H-LC,72.92,107904.0,3592.0,13941.0,0.07,3.44,12.92,16.81,27.0,0.26,0.79,20.04,X5K,ATH,METALS
71,SBIN,760.30,863.00,-11.51,M-LC,44.79,214358.0,12878.0,14341.0,-0.25,6.39,6.69,13.51,77.0,0.90,1.57,19.79,XY25,NTT,BANKS
87,UJJIVANSFB,52.77,53.00,48.19,M-SC,7.29,128736.0,-13743.0,14367.0,0.29,-9.65,11.16,0.44,247.0,-0.96,0.94,51.94,OX40N,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,HEROMOTOCO,4311.81,6039.03,-5.71,H-MC,43.23,151610.0,697.0,59750.0,-0.03,0.46,39.41,40.06,103.0,0.01,1.11,24.01,AR,ATH,AUTO


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
19,CAMPUS,294.86,393.0,-21.90,M-SC,45.83,162791.0,-4984.0,60819.0,5.69,-2.97,37.36,33.28,206.0,-0.08,1.19,26.04,XY24,NTT,FOOTWEAR
65,RELAXO,902.64,1176.0,-26.79,H-SC,96.88,66221.0,-49317.0,84306.0,3.57,-42.68,127.31,30.28,131.0,-0.58,0.48,29.37,X40N,NTT,FOOTWEAR
69,SAMMAANCAP,170.35,326.0,-168.18,M-SC,82.29,79038.0,-23172.0,116565.0,2.99,-22.67,147.48,91.37,212.0,-0.20,0.58,28.30,XY25,NTT,FINANCE
92,VIPIND,488.80,489.0,-221.42,H-SC,98.96,87545.0,-7771.0,7809.0,2.49,-8.15,8.92,0.04,157.0,-1.00,0.64,72.67,OX40N,NTT,MISC
52,JPPOWER,18.73,26.2,-13.49,L-SC,100.00,173497.0,31655.0,24914.0,2.41,22.32,14.36,39.88,262.0,1.27,1.27,75.83,XY24,NTT,POWER


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
33,GREENPANEL,375.16,537.00,210.08,M-SC,80.21,138519.0,-44559.0,123531.0,-2.04,-24.34,89.18,43.14,217.0,-0.36,1.01,30.30,XY24,NTT,MISC
12,BANKINDIA,116.91,190.00,-33.74,M-MC,10.42,175332.0,-4476.0,116894.0,-1.65,-2.49,66.67,62.52,173.0,-0.04,1.28,30.75,XR,NTT,BANKS
28,DIGITIDE,188.38,237.72,NaN,NaN,16.67,48489.0,7234.0,3574.0,-1.64,17.53,7.37,26.19,272.0,2.02,0.35,14.16,XY24,ATH,IT
95,WIPRO,243.46,420.00,-4.68,M-LC,76.04,163587.0,12642.0,96811.0,-1.47,8.38,59.18,72.51,71.0,0.13,1.19,12.68,XR,NTT,IT
74,SIEMENS,4406.90,7969.85,-6.10,X-LC,11.46,79292.0,-30880.0,119953.0,-1.45,-28.03,151.28,80.85,11.0,-0.26,0.58,16.78,AR,ATH,ELECTRICAL


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
92,VIPIND,488.80,489.0,-221.42,H-SC,98.96,87545.0,-7771.0,7809.0,2.49,-8.15,8.92,0.04,157.0,-1.00,0.64,72.67,OX40N,NTT,MISC
87,UJJIVANSFB,52.77,53.0,48.19,M-SC,7.29,128736.0,-13743.0,14367.0,0.29,-9.65,11.16,0.44,247.0,-0.96,0.94,51.94,OX40N,NTT,BANKS
79,SYMPHONY,1306.42,1306.0,-28.80,M-SC,33.33,151724.0,-19417.0,19360.0,0.42,-11.35,12.76,-0.03,191.0,-1.00,1.11,9.05,OX40N,NTT,DURABLES
46,INDIGOPNTS,1407.73,1408.0,127.12,H-SC,89.58,148874.0,-25685.0,25711.0,0.02,-14.71,17.27,0.02,133.0,-1.00,1.09,29.10,OX40N,NTT,PAINTS
81,TATAELXSI,7332.28,7332.0,-18.49,X-MC,5.21,85645.0,-17007.0,17001.0,-0.56,-16.57,19.85,-0.00,59.0,-1.00,0.63,30.19,OX40N,NTT,IT


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TCS,3794.03,4998.00,-19.34,X-LC,29.17,263258.0,-32676.0,126574.0,-0.26,-11.04,48.08,31.73,1.0,-0.26,1.92,5.37,X40,BTT,IT
48,INFY,1461.46,2275.00,-9.91,X-LC,60.42,288924.0,27323.0,118314.0,-1.20,10.44,40.95,55.67,2.0,0.23,2.11,17.19,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-13.95,X-LC,62.50,269573.0,-774.0,35395.0,-0.68,-0.29,13.13,12.81,5.0,-0.02,1.97,12.74,XY25,NTT,FMCG
17,BRITANNIA,4983.33,6446.05,17.76,X-LC,78.12,270227.0,40994.0,26293.0,-0.13,17.88,9.73,29.35,10.0,1.56,1.97,28.40,XY25,ATH,FMCG
74,SIEMENS,4406.90,7969.85,-6.10,X-LC,11.46,79292.0,-30880.0,119953.0,-1.45,-28.03,151.28,80.85,11.0,-0.26,0.58,16.78,AR,ATH,ELECTRICAL


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3133.9,NaN,NaN,95.83,78348.0,43899.0,0.0,0.24,127.43,0.00,127.43,274.0,inf,0.57,19.81,AR,ATH,ELECTRICAL
52,JPPOWER,18.73,26.2,-13.49,L-SC,100.00,173497.0,31655.0,24914.0,2.41,22.32,14.36,39.88,262.0,1.27,1.27,75.83,XY24,NTT,POWER
59,MEDANTA,1064.65,1486.0,17.58,X-SC,91.67,250610.0,45133.0,36188.0,1.55,21.96,14.44,39.58,65.0,1.25,1.83,34.30,XY24,NTT,HEALTHCARE


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,92.75,M-MC,59.38,231351.0,6389.0,161275.0,0.35,2.84,69.71,74.53,189.0,0.04,1.69,34.32,XY24,BTT,STEEL
54,KPIGREEN,497.21,731.79,29.50,H-SC,67.71,132577.0,7280.0,51838.0,1.69,5.81,39.10,47.18,143.0,0.14,0.97,65.86,MH,ATH,POWER
20,CAMS,3643.00,5250.99,4.54,H-SC,39.58,117384.0,15380.0,29639.0,0.04,15.08,25.25,44.14,126.0,0.52,0.86,33.77,X40N,ATH,MISC
15,BLUESTARCO,1646.70,2337.55,-4.05,H-SC,93.75,180100.0,15430.0,53652.0,-0.48,9.37,29.79,41.95,124.0,0.29,1.32,17.54,X40N,ATH,AC
95,WIPRO,243.46,420.00,-4.68,M-LC,76.04,163587.0,12642.0,96811.0,-1.47,8.38,59.18,72.51,71.0,0.13,1.19,12.68,XR,NTT,IT


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3133.90,NaN,NaN,95.83,78348.0,43899.0,0.0,0.24,127.43,0.00,127.43,274.0,inf,0.57,19.81,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-40.32,M-SC,63.54,196583.0,24962.0,137117.0,-0.79,14.54,69.75,94.43,201.0,0.18,1.44,45.09,XR,NTT,DURABLES
54,KPIGREEN,497.21,731.79,29.50,H-SC,67.71,132577.0,7280.0,51838.0,1.69,5.81,39.10,47.18,143.0,0.14,0.97,65.86,MH,ATH,POWER
95,WIPRO,243.46,420.00,-4.68,M-LC,76.04,163587.0,12642.0,96811.0,-1.47,8.38,59.18,72.51,71.0,0.13,1.19,12.68,XR,NTT,IT
45,INDIAMART,2327.09,4850.92,-47.14,H-SC,81.25,137800.0,14464.0,119293.0,0.48,11.73,86.57,108.45,123.0,0.12,1.01,36.70,AR,ATH,MISC


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,86.03,NaN,NaN,68.75,18838.0,-30755.0,2.0,-0.01,-62.01,0.01,-62.01,270.0,-15377.50,0.14,26.33,XY24,ATH,MISC
24,COFFEEDAY,59.14,80.00,-55.73,L-SC,20.83,66874.0,-46675.0,86729.0,1.40,-41.11,129.69,35.27,265.0,-0.54,0.49,63.67,XR,NTT,HOTELS
32,ENRIN,1377.95,3133.90,NaN,NaN,95.83,78348.0,43899.0,0.0,0.24,127.43,0.00,127.43,274.0,inf,0.57,19.81,AR,ATH,ELECTRICAL
92,VIPIND,488.80,489.00,-221.42,H-SC,98.96,87545.0,-7771.0,7809.0,2.49,-8.15,8.92,0.04,157.0,-1.00,0.64,72.67,OX40N,NTT,MISC
58,MASFIN,326.60,399.50,-17.34,H-SC,77.08,93900.0,-4080.0,25954.0,0.14,-4.16,27.64,22.32,145.0,-0.16,0.69,36.19,XR,ATH,FINANCE


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-13.49,L-SC,100.00,173497.0,31655.0,24914.0,2.41,22.32,14.36,39.88,262.0,1.27,1.27,75.83,XY24,NTT,POWER
92,VIPIND,488.80,489.00,-221.42,H-SC,98.96,87545.0,-7771.0,7809.0,2.49,-8.15,8.92,0.04,157.0,-1.00,0.64,72.67,OX40N,NTT,MISC
3,ALKYLAMINE,2347.98,4546.37,7.26,X-SC,97.92,99855.0,-1108.0,95641.0,-0.12,-1.10,95.78,93.63,67.0,-0.01,0.73,48.73,SR,ATH,CHEMICALS
65,RELAXO,902.64,1176.00,-26.79,H-SC,96.88,66221.0,-49317.0,84306.0,3.57,-42.68,127.31,30.28,131.0,-0.58,0.48,29.37,X40N,NTT,FOOTWEAR
32,ENRIN,1377.95,3133.90,NaN,NaN,95.83,78348.0,43899.0,0.0,0.24,127.43,0.00,127.43,274.0,inf,0.57,19.81,AR,ATH,ELECTRICAL


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.96
1,25,41.94
2,50,70.61


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.72
LC    33.68
MC    21.54
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     31.09
X40      12.88
XY25     12.07
X40N     11.93
XR        9.94
OX40N     8.22
AR        8.16
MH        1.70
X5K       1.41
X200      1.39
SR        1.21
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.55
M-SC    15.93
H-LC    14.01
X-LC    12.84
H-MC     9.59
X-MC     7.57
M-LC     5.79
X-SC     4.58
M-MC     4.03
L-SC     2.66
L-LC     1.04
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.24,-0.26,33.79
IT,11.97,-10.72,65.14
FINANCE,8.55,-11.52,51.99
BANKS,7.61,-9.27,53.86
MISC,5.89,-19.38,74.97
PAINTS,5.67,-12.46,34.62
ELECTRICAL,5.45,1.24,45.41
HEALTHCARE,4.74,0.83,28.14
AUTO,4.31,-14.68,65.11


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2632327.0,26
XR,1096632.0,13
AR,1030306.0,10
X40,690851.0,10
X40N,529752.0,11
XY25,467037.0,8
OX40N,306587.0,11
SR,178099.0,2
MH,83652.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1969440.0,20
M-SC,1763113.0,21
H-LC,621673.0,12
X-LC,551755.0,8
H-MC,521293.0,8
M-MC,417673.0,3
X-MC,415915.0,6
X-SC,292146.0,5
M-LC,268471.0,4


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      890285.0      6
M-SC       XY24      801870.0      7
H-SC       AR        522688.0      3
M-SC       XR        340529.0      4
M-MC       XY24      300779.0      2
H-MC       XY24      295696.0      3
X-LC       X40       280330.0      3
H-SC       XR        217339.0      3
           X40N      201714.0      4
H-LC       X40N      199146.0      4
M-SC       AR        187978.0      2
           OX40N     164829.0      5
H-LC       X40       158389.0      3
M-LC       XY24      157319.0      2
X-MC       X40       156838.0      2
L-SC       XR        149846.0      2
X-SC       XR        121136.0      1
X-LC       AR        119953.0      1
M-MC       XR        116894.0      1
M-SC       XY25      116565.0      1
X-MC       XY25      105385.0      1
X-LC       XY25      101372.0      3
M-LC       XR         96811.0      1
X-SC       SR         95641.0      1
H-SC       OX40N      85576.0      3
X-MC       XY24       83552.0      1
M-SC       SR         82458.0      1
H-MC       XY25       81970.0      1
H-LC       AR         80816.0      1
           X5K        75615.0      2
           X200       69559.0      1
M-SC       X40        68884.0      1
H-MC       AR         59750.0      1
L-SC       AR         59121.0      1
L-MC       XR         54077.0      1
X-MC       X40N       53139.0      1
H-SC       MH         51838.0      1
X-LC       X40N       50100.0      1
L-LC       XY25       47404.0      1
X-SC       OX40N      39181.0      2
H-LC       XY24       38148.0      1
X-SC       XY24       36188.0      1
H-MC       MH         31814.0      1
           X40        26410.0      1
           X40N       25653.0      1
L-SC       XY24       24914.0      1
X-MC       OX40N      17001.0      1
M-LC       XY25       14341.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
